<a href="https://colab.research.google.com/github/Priya1881/NLP/blob/main/NewsClassificationUsingBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow_text

     |████████████████████████████████| 4.9 MB 5.2 MB/s 


In [3]:
!pip install tensorflow_hub

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [20]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/bilstm_news.csv")
df.head()

,Unnamed: 0,Text,Category
0,0,worldcom ex-boss launches defence lawyers defe...,business
1,1,german business confidence slides german busin...,business
2,2,bbc poll indicates economic gloom citizens in ...,business
3,3,lifestyle governs mobile choice faster bett...,tech
4,4,enron bosses in $168m payout eighteen former e...,business


In [21]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [24]:
data=df.copy()

In [25]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['Category']=encoder.fit_transform(data['Category'])

In [8]:
df['Text'] = (data['Text']).apply(lambda row: row.strip())

In [27]:
df['Category'].value_counts()

5    346
0    336
4    300
2    300
3    274
1    273
6    261
Name: Category, dtype: int64

In [10]:
labels=tf.keras.utils.to_categorical(df['Category'].values,num_classes=7)



In [11]:
labels

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Text'],labels,test_size=0.3)

Now lets import BERT model and get embeding vectors for few sample statements


In [5]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")


Build Model
There are two types of models you can build in tensorflow.

(1) Sequential (2) Functional

So far we have built sequential model. But below we will build functional model. More information on these two is here: https://becominghuman.ai/sequential-vs-functional-model-in-keras-20684f766057

In [14]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(7, activation='softmax', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [15]:
METRICS = [
      tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=METRICS)

In [ ]:
model.summary()

In [17]:
model.fit(X_train,y_train, epochs=12)

Epoch 1/12
46/46 [==============================] - 619s 13s/step - loss: 1.7801 - accuracy: 0.3021 - precision: 0.6562 - recall: 0.0144
Epoch 2/12
46/46 [==============================] - 608s 13s/step - loss: 1.4629 - accuracy: 0.4347 - precision: 0.6759 - recall: 0.0670
Epoch 3/12
46/46 [==============================] - 607s 13s/step - loss: 1.2780 - accuracy: 0.5284 - precision: 0.7785 - recall: 0.1586
Epoch 4/12
46/46 [==============================] - 609s 13s/step - loss: 1.1622 - accuracy: 0.5871 - precision: 0.7962 - recall: 0.2269
Epoch 5/12
46/46 [==============================] - 607s 13s/step - loss: 1.0633 - accuracy: 0.6446 - precision: 0.8267 - recall: 0.3001
Epoch 6/12
46/46 [==============================] - 622s 14s/step - loss: 0.9711 - accuracy: 0.7061 - precision: 0.8576 - recall: 0.3664
Epoch 7/12
46/46 [==============================] - 613s 13s/step - loss: 0.9169 - accuracy: 0.7348 - precision: 0.8699 - recall: 0.4252
Epoch 8/12
46/46 [=======================

In [18]:
model.evaluate(X_test, y_test)

20/20 [==============================] - 261s 13s/step - loss: 0.6992 - accuracy: 0.8262 - precision: 0.9169 - recall: 0.6507


[0.6991900205612183,
 0.8261563181877136,
 0.9168539047241211,
 0.6507176756858826]

In [3]:
#!mkdir -p /content/drive/MyDrive/saved_model
#model.save('/content/drive/MyDrive/saved_model/my_model')

In [ ]:
import tensorflow as tf
import tensorflow_text as text # this has to be mentioned for loading the model 

new_model = tf.keras.models.load_model('/content/drive/MyDrive/saved_model/my_model')

# Check its architecture
new_model.summary()

In [19]:
 Evaluate the restored model
loss, accuracy,precision,recall = new_model.evaluate(X_test, y_test, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))
print(new_model.predict(X_test).shape)

In [46]:
print(new_model.predict(X_test).shape)

(627, 7)


In [10]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

input=get_sentence_embeding([
    "Shares of One97 Communications Limited", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)


In [6]:
business="The State Bank of India (SBI) has urged its customers to link PAN with Aadhaar for seamless banking services and to avoid any confusion. Check the details here."
sport="Bhavin, are you up for a volleybal game tomorrow?"
tech="Airtel vs Vi Recharge Plans: Like Airtel, Vi’s upgraded base prepaid plans will also start from Rs 99, with the introduction of tariff hikes. Some plans from Vi are marginally cheaper than Airtel’s upgraded plans."
ent="Scarlett Johansson received the American Cinematheque Award recently. She has become the 35th recipient of the award."
science="Nasa has launched the Dart mission that will crash into an asteroid in an attempt to demonstrate a new planetary defence system."
politics="Congress MLA from Rae Bareli in Uttar Pradesh, Aditi Singh is likely to join the Bharatiya Janata Party (BJP) later today."


In [16]:
prediction=new_model.predict([business])
prediction

array([[0.05459446, 0.02856889, 0.23918629, 0.04781201, 0.56023693,
        0.02407464, 0.04552682]], dtype=float32)

In [17]:
import numpy as np
#converting the one hot vector output to a linear numpy array.
pred_class = np.argmax(prediction, axis = 1)

In [18]:
if pred_class[0] == 0: 
   category='Business'
if pred_class[0] == 1:
    category='Entertainment'
if pred_class[0] == 2:
    category='Health'
if pred_class[0] == 3:
    category='Politics'
if pred_class[0] == 4:
    category='Science'
if pred_class[0] == 5:
    category='Sports'
if pred_class[0] == 6: 
    category='Technology' 

print(category)


Science
